In [1]:
!pip install "git+https://github.com/huggingface/peft"
!pip install "evaluate==0.4.0" loralib --upgrade --quiet
!pip install rouge-score tensorboard py7zr
!pip install -U datasets
!pip install tf-keras
!pip install trl
!pip install triton
!pip install "torchvision -index-url https://download.pytorch.org/whl/cu118"                                                                           
!pip install "torchaudio -index-url https://download.pytorch.org/whl/cu118"  
!pip install "bitsandbytes>=0.39.0"
!pip install "accelerate>=0.16.0,<1"
!pip install -q -U peft
!pip install "transformers[torch]>=4.28.1"
!pip install unsloth
!pip install --no-deps "xformers<0.0.27"


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/peft to c:\users\james\appdata\local\temp\pip-req-build-98_foaro
  Resolved https://github.com/huggingface/peft to commit e8ba7de5732e050a8f061cde555c02ab575f7529
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft 'C:\Users\james\AppData\Local\Temp\pip-req-build-98_foaro'


In [ ]:
import json 
data = ""
data_path = r"C:\Users\james\Documents\tools\GenerativeExtremism\data.json"
with open(data_path) as json_file:
    data = json.load(json_file)

processed_data = []
for convo in data:
    for post in data[convo]:
        human_label = data[convo][post]["human_label"][0]
        gpt4o_label = data[convo][post]["gpt40"]["label"]
        gpt4o_reasoning = data[convo][post]["gpt40"]["reasoning"]
        # Validate data by saying gpt4o result must be same as human
        if human_label != gpt4o_label:
            continue
        
        user = f"User {data[convo][post]["id"]}"
        full_response = f"The post is {gpt4o_label}. {gpt4o_reasoning}".replace(user,"The author")
        messages = post.split(" - ")
        for message in messages:
            processed_data.append({"message":message,"reasoning":full_response})

print(processed_data)

[{'message': '', 'reasoning': 'The post is Not Extremist. The author posted a YouTube link without any additional contextual information. There is no promotion of violence, hatred, or intolerance in this post.'}, {'message': 'https://youtu.be/U_tnBgCmpOQ', 'reasoning': 'The post is Not Extremist. The author posted a YouTube link without any additional contextual information. There is no promotion of violence, hatred, or intolerance in this post.'}, {'message': '', 'reasoning': 'The post is Not Extremist. The authors posts are discussing the shutdown of a subreddit and suggesting that it is being done to prevent a political outcome. However, there is no promotion of violence, hatred, or intolerance, nor does it aim to negate fundamental rights or create a permissive environment for extremist behavior.'}, {'message': "Y'all know that /r/td got shut down, right?", 'reasoning': 'The post is Not Extremist. The authors posts are discussing the shutdown of a subreddit and suggesting that it i

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

def load_model(model_name: str, device: str):
    """
    Load the pretrained language model and tokenizer.

    Args:
        model_name: Name of the pretrained model.
        device: Device to load the model onto.

    Returns:
        model: Loaded language model.
        tokenizer: Loaded tokenizer.
    """
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
    return model, tokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
model, tokenizer = load_model(model_name, device)

C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\quantizers\auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is qu

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Split the data into train and temp (temp will be split into test and validation)
train_test_split_ratio = 0.8  # 80% for train, 20% for temp
train_data, temp_data = train_test_split(processed_data, test_size=1 - train_test_split_ratio)

# Split the temp data into test and validation
test_val_split_ratio = 0.5  # 50% of temp data for test, 50% for validation
test_data, val_data = train_test_split(temp_data, test_size=test_val_split_ratio)

# Convert the split lists back to Dataset objects
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)
val_dataset = Dataset.from_list(val_data)

# Create DatasetDict object
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': val_dataset
})

# Print the results
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['message', 'reasoning'],
        num_rows: 1133
    })
    test: Dataset({
        features: ['message', 'reasoning'],
        num_rows: 142
    })
    validation: Dataset({
        features: ['message', 'reasoning'],
        num_rows: 142
    })
})


In [ ]:
from datasets import concatenate_datasets
import numpy as np

# Max input sequence length post-tokenization.
max_input_length = 512
max_target_length = 50

def preprocess_function(processed_data):
    inputs = [ex for ex in processed_data['message']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setting up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(processed_data['reasoning'], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/1133 [00:00<?, ? examples/s]C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 142/142 [00:00<00:00, 10848.06 examples/s]


In [ ]:
!pip install -U unsloth

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from unsloth import FastLanguageModel
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

ModuleNotFoundError: No module named 'triton'

In [ ]:
from transformers import TrainingArguments, Trainer
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
max_seq_length = 2048
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
)
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=training_args,
)

trainer.train()

C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\trl\trainer\sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


ValueError: You cannot perform fine-tuning on purely quantized models. Please attach trainable adapters on top of the quantized model to correctly perform fine-tuning. Please see: https://huggingface.co/docs/transformers/peft for more details

In [ ]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")
def evaluate_peft_model(sample, max_target_length=50):
    # Summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    
    # Decoding evaluation sample
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)
    return prediction, labels

# Loading test dataset
test_dataset = load_from_disk("data/eval/").with_format("torch")


# Predictions
predictions, references = [], []
for sample in tqdm(test_dataset):
    p, l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)


# Printing metrics
rouge = metric.compute(predictions=predictions, references=references, use_stemmer=True)
# Print results
print(f"ROUGE-I: {rouge['rouge1']*100:.2f}%")
print(f"ROUGE-II: {rouge['rouge2']*100:.2f}%")
print(f"ROUGEL: {rouge['rougeL']*100:.2f}%")
print(f"ROUGELsum: {rouge['rougeLsum']*100:.2f}%")